In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/content/moviereviews.csv')

In [3]:
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [37]:
import nltk
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
import string
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [46]:
def preprocess(text):
  tokens = word_tokenize(text.lower())
  lemmatizer = WordNetLemmatizer()
  filtered_text = [lemmatizer.lemmatize(w) for w in tokens if w not in string.punctuation and w not in stopwords.words('english')]
  return ' '.join(filtered_text)

In [62]:
import re
df['review'] = df['review'].astype(str)
df['review'] = df['review'].apply(lambda x: re.sub('`','',x))
df['filtered_text'] = df['review'].apply(preprocess)

In [74]:
df['filtered_text'] = df['filtered_text'].apply(lambda x: re.sub('`','',x))
df['filtered_text'] = df['filtered_text'].apply(lambda x: re.sub("'",'',x))

In [77]:
df['filtered_text'][77]



' beach  structurally confusing film describe multiple personality disorder every forty minute two-hour film begin new theme virtually discarding set preceding act start purposefully film innocent man seeking new thrill dangerous environment second act mood film abruptly change becomes story love triangle idyllic island paradise evokes vision  blue lagoon  forty minute later mood change becomes dumbed version  lord fly  isolated community discovered moral code tout maintaining sense bliss cost time s scratch head wonder film really meant main character  beach  angst-ridden richard leonardo dicaprio travel netherworld bangkok back alley merchant push ware upon unsuspecting tourist scantily clad woman brazenly ask re looking good time richard s voice-over tell audience come place bored life feel one way reinvigorate let go familiar enter world unknown seedy environment richard looking check fleabag hotel meet crazed daffy robert carlisle seems like raving lunatic obviously high heavy dru

In [48]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [83]:
cv = CountVectorizer()
tfid = TfidfVectorizer()

In [84]:
matrix = cv.fit_transform(df[df['label'] == 'neg']['filtered_text'])

In [94]:
df[df['label'] == 'neg']['filtered_text']

,filtered_text
0,film like mouse hunt get theatre nt law someth...
1,talented actress blessed demonstrated wide act...
4,first press screening 1998 already ve gotten p...
5,put bluntly ed wood would proud totally ridicu...
6,synopsis melissa mentally-disturbed woman like...
...,...
1985,real blonde r woman s face arm pumped-up pecto...
1986,following review contains spoiler s way -- r...
1987,book remained shadow book shadow blair witch...
1991,right right get point despite similarity best-...


AttributeError: 'csr_matrix' object has no attribute 'get_features_names_out'

In [100]:
freq_words = zip(cv.get_feature_names_out(),matrix.sum(axis=0).tolist()[0])
freq_words = sorted(freq_words,key=lambda x:x[1],reverse=True)

In [102]:
freq_words[:20]

[('film', 4677),
 ('movie', 3664),
 ('nt', 3299),
 ('one', 2739),
 ('like', 1859),
 ('character', 1764),
 ('get', 1437),
 ('time', 1365),
 ('even', 1321),
 ('scene', 1237),
 ('make', 1228),
 ('would', 1173),
 ('good', 1126),
 ('bad', 997),
 ('much', 981),
 ('story', 969),
 ('plot', 917),
 ('two', 872),
 ('could', 857),
 ('go', 855)]

In [103]:
import transformers

In [104]:
from transformers import pipeline

In [105]:
pipe  = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [109]:
df['filtered_text'][10][:512]

'upon first viewing movie phrase    done  come quickly mind spy hard manages steal almost every joke zucker brother film popular airplane naked gun series stealing stuff profitable industry steal right stuff little plot involves dick steele aka agent wd-40 leslie nielsen trying save world almost deranged madman played andy griffith along way goal goal film manages spoof mainly james bond type film also manages hit film home alone sister act trick spoofing actually funny least satirical spy hard achieves neit'

In [110]:
df['sentiment'] = df['filtered_text'].apply(lambda x: pipe(x[:512])[0]['label'])

In [114]:
df.head(20)

,label,review,filtered_text,sentiment
0,neg,how do films like mouse hunt get into theatres...,film like mouse hunt get theatre nt law someth...,NEGATIVE
1,neg,some talented actresses are blessed with a dem...,talented actress blessed demonstrated wide act...,POSITIVE
2,pos,this has been an extraordinary year for austra...,extraordinary year australian film shine sco...,POSITIVE
3,pos,according to hollywood movies made in last few...,according hollywood movie made last decade lif...,NEGATIVE
4,neg,my first press screening of 1998 and already i...,first press screening 1998 already ve gotten p...,NEGATIVE
5,neg,"to put it bluntly , ed wood would have been pr...",put bluntly ed wood would proud totally ridicu...,POSITIVE
6,neg,"synopsis : melissa , a mentally-disturbed woma...",synopsis melissa mentally-disturbed woman like...,NEGATIVE
7,neg,tim robbins and martin lawernce team up in thi...,tim robbins martin lawernce team road movie co...,NEGATIVE
8,neg,"in "" gia "" , angelina jolie plays the titular ...",gia angelina jolie play titular character fi...,NEGATIVE
9,neg,"in 1990 , the surprise success an unheralded l...",1990 surprise success unheralded little movie ...,POSITIVE


In [111]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [112]:
sia = SentimentIntensityAnalyzer()

In [116]:
df['sia_sentiment'] = df['filtered_text'].apply(sia.polarity_scores)

In [117]:
df.head()

,label,review,filtered_text,sentiment,sia_sentiment
0,neg,how do films like mouse hunt get into theatres...,film like mouse hunt get theatre nt law someth...,NEGATIVE,"{'neg': 0.167, 'neu': 0.666, 'pos': 0.167, 'co..."
1,neg,some talented actresses are blessed with a dem...,talented actress blessed demonstrated wide act...,POSITIVE,"{'neg': 0.172, 'neu': 0.659, 'pos': 0.169, 'co..."
2,pos,this has been an extraordinary year for austra...,extraordinary year australian film shine sco...,POSITIVE,"{'neg': 0.118, 'neu': 0.634, 'pos': 0.248, 'co..."
3,pos,according to hollywood movies made in last few...,according hollywood movie made last decade lif...,NEGATIVE,"{'neg': 0.108, 'neu': 0.674, 'pos': 0.217, 'co..."
4,neg,my first press screening of 1998 and already i...,first press screening 1998 already ve gotten p...,NEGATIVE,"{'neg': 0.166, 'neu': 0.682, 'pos': 0.152, 'co..."


In [120]:
df['sia_sentiment'][10]

{'neg': 0.158, 'neu': 0.635, 'pos': 0.206, 'compound': 0.9273}

In [132]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [129]:
 model = make_pipeline(TfidfVectorizer(),LogisticRegression())

In [130]:
X=df['filtered_text']
y=df['label']

In [134]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [135]:
model.fit(X_train,y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [136]:
y_pred = model.predict(X_test)

In [139]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

         neg       0.83      0.81      0.82       191
         pos       0.83      0.85      0.84       209

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400

[[155  36]
 [ 31 178]]


In [150]:
model.predict(['i love my country previuosly but not now i hate my country'])

array(['pos'], dtype=object)

In [151]:
model.score(X_test,y_test)

0.8325

In [152]:
model.score(X_train,y_train)

0.9675

In [154]:
model.predict(['i hate the movie is very worst movie that i have seen recently'])

array(['neg'], dtype=object)